<a href="https://colab.research.google.com/github/msamwelmollel/swahili-prompt-translation-for-swahili/blob/main/Uploading_prompts_to_be_translated_to_an_Argilla_Space.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%pip install huggingface_hub argilla datasets openai -qqq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 420.4/420.4 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 20.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 262.9/262.9 kB 18.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.9/75.9 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 41.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 2.7 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
chex 0.1.8

In [11]:
import json

In [12]:
SPACE_ID =  "DIBT-Swahili/prompt-translation-for-Swahili"
HOMEPAGE_URL = "https://dibt-swahili-prompt-translation-for-swahili.hf.space"
LANGUAGE = None, # i.e. "Swahili"

HF_ORG_NAME = 'DIBT-Swahili' # update with the ID of the org you just created
LANGUAGE = 'Swahili' # update this with the language you will work on
to_id =  f"{HF_ORG_NAME}/prompt-translation-for-{LANGUAGE}"
to_id

'DIBT-Swahili/prompt-translation-for-Swahili'

In [13]:
# from huggingface_hub import restart_space
# restart_space(to_id, factory_reboot=True)

SpaceRuntime(stage='BUILDING', hardware=None, requested_hardware='cpu-upgrade', sleep_time=172800, storage='small', raw={'stage': 'BUILDING', 'hardware': {'current': None, 'requested': 'cpu-upgrade'}, 'storage': 'small', 'gcTimeout': 172800, 'replicas': {'current': 1, 'requested': 1}, 'devMode': False})

In [17]:
assert SPACE_ID and HOMEPAGE_URL and LANGUAGE, "Please set SPACE_ID and HOMEPAGE_URL to your space ID and homepage URL"

In [18]:
from huggingface_hub import space_info

assert space_info(SPACE_ID).runtime.storage.get("current") == "small", "Please ensure you have setup persistent storage for your space. Please see steps above"

In [15]:
import argilla as rg

OWNER_API_KEY = "b6f4a5b1-1c74-4927-b876-34ffe941706d" # if you haven't setup the secret this is the default owner api key
assert OWNER_API_KEY is not None, "Please set OWNER_API_KEY to the API token you just set in the Space settings"

rg.init(api_url=HOMEPAGE_URL, api_key=OWNER_API_KEY)

/usr/local/lib/python3.10/dist-packages/argilla/client/client.py:178: UserWarning: No workspace configuration was detected. To work with Argilla datasets, specify a valid workspace name on `rg.init` or set it up through the `rg.set_workspace` function.
  warnings.warn(


In [29]:
from datasets import load_dataset

# load the dataset from the Hub
ds = load_dataset('DIBT/prompts_ranked_multilingual_benchmark')

In [30]:
# create the dataset with a pre-built template
argilla_ds = rg.FeedbackDataset.for_translation(
    use_markdown=True,
    guidelines=None,
    metadata_properties=None,
    vectors_settings=None,
)
argilla_ds

FeedbackDataset(
   fields=[TextField(name='source', title='Source', required=True, type='text', use_markdown=True)]
   questions=[TextQuestion(name='target', title='Target', description='Translate the text.', required=True, type='text', use_markdown=True)]
   guidelines=This is a translation dataset that contains texts. Please translate the text in the text field.)
   metadata_properties=[])
   vectors_settings=[])
)

In [31]:
# create records
records = []
for row in ds["train"]:
    record = rg.FeedbackRecord(
        fields={"source": row["prompt"]},
        metadata=json.loads(row["metadata"]),
        external_id=row["row_idx"],
    )
    records.append(record)

In [32]:
# add records to the dataset
argilla_ds.add_records(records)

In [35]:
# push the dataset to Argilla
argilla_ds.push_to_argilla(f"DIBT Translation for {LANGUAGE}", workspace="admin")

Output()

INFO:argilla.client.feedback.dataset.local.mixins:✓ Dataset succesfully pushed to Argilla
INFO:argilla.client.feedback.dataset.local.mixins:RemoteFeedbackDataset(
   id=3579959c-c8bb-4f01-8bbd-fcd9ca36a20e
   name=DIBT Translation for Swahili
   workspace=Workspace(id=f151c9eb-3bbb-4a7e-9688-14fd7fe765db, name=admin, inserted_at=2024-03-28 10:26:39.534568, updated_at=2024-03-28 10:26:39.534568)
   url=https://dibt-swahili-prompt-translation-for-swahili.hf.space/dataset/3579959c-c8bb-4f01-8bbd-fcd9ca36a20e/annotation-mode
   fields=[RemoteTextField(id=UUID('57e0c14a-315b-4d86-ae80-a74f2e9bd3c9'), client=None, name='source', title='Source', required=True, type='text', use_markdown=True)]
   questions=[RemoteTextQuestion(id=UUID('f73079a4-c69f-46d0-b4df-9f1d10d19750'), client=None, name='target', title='Target', description='Translate the text.', required=True, type='text', use_markdown=True)]
   guidelines=This is a translation dataset that contains texts. Please translate the text in th

RemoteFeedbackDataset(
   id=3579959c-c8bb-4f01-8bbd-fcd9ca36a20e
   name=DIBT Translation for Swahili
   workspace=Workspace(id=f151c9eb-3bbb-4a7e-9688-14fd7fe765db, name=admin, inserted_at=2024-03-28 10:26:39.534568, updated_at=2024-03-28 10:26:39.534568)
   url=https://dibt-swahili-prompt-translation-for-swahili.hf.space/dataset/3579959c-c8bb-4f01-8bbd-fcd9ca36a20e/annotation-mode
   fields=[RemoteTextField(id=UUID('57e0c14a-315b-4d86-ae80-a74f2e9bd3c9'), client=None, name='source', title='Source', required=True, type='text', use_markdown=True)]
   questions=[RemoteTextQuestion(id=UUID('f73079a4-c69f-46d0-b4df-9f1d10d19750'), client=None, name='target', title='Target', description='Translate the text.', required=True, type='text', use_markdown=True)]
   guidelines=This is a translation dataset that contains texts. Please translate the text in the text field.
   metadata_properties=[]
   vectors_settings=[]
)

In [16]:
# rg.delete('DIBT Translation for Swahili', workspace='admin')

<ipython-input-16-965c42620148>:1: UserWarning: Removed dataset was a `FeedbackDataset`. It's recommended to use `rg.FeedbackDataset.from_argilla(name='DIBT Translation for Swahili', workspace='admin').delete()` instead.
  rg.delete('DIBT Translation for Swahili', workspace='admin')


In [36]:
# !pip install 'transformers[torch]'

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.1/290.1 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 17.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 28.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 26.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 1.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 1.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.0/166.0 MB 923.4 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.1/99.1 kB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━

In [37]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
import torch

model_path = "facebook/nllb-200-distilled-600M"
tokenizer = AutoTokenizer.from_pretrained(model_path, src_lang="eng_Latn")
model = AutoModelForSeq2SeqLM.from_pretrained(model_path)

# Check if a GPU is available
device = "cuda" if torch.cuda.is_available() else "cpu"

# Move the model and tokenizer to the GPU if available
model = model.to(device)

tokenizer_config.json:   0%|          | 0.00/564 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/4.85M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.3M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/3.55k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/846 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.46G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/189 [00:00<?, ?B/s]

In [38]:
def open_translate(texts, trg_lang):
    if isinstance(texts, str):  # If a single text is provided, convert it to a list
        texts = [texts]

    inputs = tokenizer(texts, return_tensors="pt", padding=True, truncation=True)
    translated_tokens = model.generate(
        **inputs.to(device), forced_bos_token_id=tokenizer.lang_code_to_id[trg_lang]
    )
    translations = tokenizer.batch_decode(translated_tokens, skip_special_tokens=True)

    if len(translations) == 1:
        return translations[0]
    else:
        return translations


example = "We will first start with an example of a translation pipeline with open source models. Even though these models are able to run on CPU it is highly recommended to use a GPU in order to speed up inference."
open_translate(example, "swh_Latn")

the `lang_code_to_id` attribute is deprecated. The logic is natively handled in the `tokenizer.adder_tokens_decoder` this attribute will be removed in `transformers` v4.38


'Kwanza tutaanza na mfano wa bomba la tafsiri na mifano ya chanzo wazi. Ingawa mifano hii ni uwezo wa kukimbia juu ya CPU ni sana ilipendekeza kutumia GPU ili kuharakisha inference.'

In [39]:
argilla_ds = rg.FeedbackDataset.from_argilla(f"DIBT Translation for {LANGUAGE}", workspace="admin")
argilla_ds

RemoteFeedbackDataset(
   id=3579959c-c8bb-4f01-8bbd-fcd9ca36a20e
   name=DIBT Translation for Swahili
   workspace=Workspace(id=f151c9eb-3bbb-4a7e-9688-14fd7fe765db, name=admin, inserted_at=2024-03-28 10:26:39.534568, updated_at=2024-03-28 10:26:39.534568)
   url=https://dibt-swahili-prompt-translation-for-swahili.hf.space/dataset/3579959c-c8bb-4f01-8bbd-fcd9ca36a20e/annotation-mode
   fields=[RemoteTextField(id=UUID('57e0c14a-315b-4d86-ae80-a74f2e9bd3c9'), client=None, name='source', title='Source', required=True, type='text', use_markdown=True)]
   questions=[RemoteTextQuestion(id=UUID('f73079a4-c69f-46d0-b4df-9f1d10d19750'), client=None, name='target', title='Target', description='Translate the text.', required=True, type='text', use_markdown=True)]
   guidelines=This is a translation dataset that contains texts. Please translate the text in the text field.
   metadata_properties=[]
   vectors_settings=[]
)

In [40]:
from tqdm.auto import tqdm

In [ ]:
altered_records = []
for rec in tqdm(argilla_ds.records):
    rec.suggestions = [
        {
            "question_name": "target",
            "value": open_translate(rec.fields["source"], "swh_Latn")
        }
    ]
    altered_records.append(rec)

  0%|          | 0/501 [00:00<?, ?it/s]

In [ ]:
argilla_ds.update_records(altered_records)